^C


In [1]:
import cv2
import csv
import mediapipe as mp
import face_recognition
import numpy as np
import copy
import math
import itertools
from model import KeyPointClassifier
from pprint import pprint
mp_face_detection = mp.solutions.face_detection
mp_hands = mp.solutions.hands
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [2]:
 with open('model/keypoint_classifier/keypoint_classifier_label.csv',
              encoding='utf-8-sig') as f:
        keypoint_classifier_labels = csv.reader(f)
        keypoint_classifier_labels = [
            row[0] for row in keypoint_classifier_labels
        ]
print(keypoint_classifier_labels)
keypoint_classifier = KeyPointClassifier()

['Open', 'Close', 'Pointer', 'OK']


In [3]:
def calc_landmark_list(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]

    landmark_point = []

    # Keypoint
    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)
        # landmark_z = landmark.z

        landmark_point.append([landmark_x, landmark_y])

    return landmark_point

def pre_process_landmark(landmark_list):
    temp_landmark_list = copy.deepcopy(landmark_list)

    # Convert to relative coordinates
    base_x, base_y = 0, 0
    for index, landmark_point in enumerate(temp_landmark_list):
        if index == 0:
            base_x, base_y = landmark_point[0], landmark_point[1]

        temp_landmark_list[index][0] = temp_landmark_list[index][0] - base_x
        temp_landmark_list[index][1] = temp_landmark_list[index][1] - base_y

    # Convert to a one-dimensional list
    temp_landmark_list = list(
        itertools.chain.from_iterable(temp_landmark_list))

    # Normalization
    max_value = max(list(map(abs, temp_landmark_list)))

    def normalize_(n):
        return n / max_value

    temp_landmark_list = list(map(normalize_, temp_landmark_list))

    return temp_landmark_list

In [9]:
rewant_image = face_recognition.load_image_file("rewant.jpg")

rewant_face_encoding = face_recognition.face_encodings(rewant_image)[0]

sanjay_image = face_recognition.load_image_file("sanjay.jpg")

sanjay_face_encoding = face_recognition.face_encodings(sanjay_image)[0]

samikshya_image = face_recognition.load_image_file("samikshya.jpg")

samikshya_face_encoding = face_recognition.face_encodings(samikshya_image)[0]
# print(rewant_face_encoding)

known_face_encodings = [
    rewant_face_encoding,
    samikshya_face_encoding,
    sanjay_face_encoding
]

known_face_names = [
    "Rewant",
    "Samikshya",
    "Sanjay"
]

face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

In [8]:
# For webcam input:
cap = cv2.VideoCapture(0)
with mp_face_detection.FaceDetection(
    min_detection_confidence=0.5) as face_detection, mp_hands.Hands(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5, max_num_hands=10) as hands, mp_holistic.Holistic(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as holistic:
 while cap.isOpened():
    success, image = cap.read()
#     print(image.shape[1])
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # Flip the image horizontally for a later selfie-view display, and convert
    # the BGR image to RGB.
    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
    
    
    if process_this_frame:
        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(image)
#         print(face_locations[0])
        face_landmarks_list = face_recognition.face_landmarks(image)
#         print(face_landmarks_list)
        if(face_landmarks_list!=[]):
#             print(face_landmarks_list[0]['nose_tip'][0][0])
#             print(face_landmarks_list[0]['nose_tip'][0][1])
            nose_x = face_landmarks_list[0]['nose_tip'][0][0]
            nose_y = face_landmarks_list[0]['nose_tip'][0][1]
        face_encodings = face_recognition.face_encodings(image, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            # # If a match was found in known_face_encodings, just use the first one.
            # if True in matches:
            #     first_match_index = matches.index(True)
            #     name = known_face_names[first_match_index]

            # Or instead, use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            face_names.append(name)

    process_this_frame = not process_this_frame
    
    
    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    face_results = face_detection.process(image)
    hand_results = hands.process(image)
#     holistic_results = holistic.process(image)

    # Draw the face detection annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

#     mp_drawing.draw_landmarks(
#         image, holistic_results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
#     mp_drawing.draw_landmarks(
#         image, holistic_results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
#     mp_drawing.draw_landmarks(
#         image, holistic_results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
#     print(holistic_results.left_hand_landmarks)
#     print(holistic_results.right_hand_landmarks)
#     if hand_results.multi_hand_landmarks:
#         print((hand_results.multi_hand_landmarks[0]))
#     print(vars(hand_results))
#     print(face_results)
    
    if face_results.detections:
      for detection in face_results.detections:
#         print(detection.location_data.relative_bounding_box)
#         print('Nose tip:')
#         print(mp_face_detection.get_key_point(
#           detection, mp_face_detection.FaceKeyPoint.LEFT_EYE))
        mp_drawing.draw_detection(image, detection)
    if hand_results.multi_hand_landmarks:
      for hand_landmarks in hand_results.multi_hand_landmarks:
        hand_x = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_MCP].x * image.shape[1]
        hand_y = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_MCP].y * image.shape[0]
        dist = math.sqrt(pow((hand_x-nose_x),2)+pow((hand_y-nose_y),2))
#         print('distance is')
#         print(dist)
#         print(hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_MCP].y * image.shape[0])
#         print(hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_MCP].x * image.shape[1])
#         print(hand_landmarks)
        
        landmark_list = calc_landmark_list(image, hand_landmarks)

        # Conversion to relative coordinates / normalized coordinates
        pre_processed_landmark_list = pre_process_landmark(landmark_list)
        hand_sign_id = keypoint_classifier(pre_processed_landmark_list)
        print(keypoint_classifier_labels[hand_sign_id])
        mp_drawing.draw_landmarks(
            image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
    cv2.imshow('Face and Hand Detection', image)
#     print(cv2.getWindowImageRect('Face and Hand Detection'))
    if cv2.waitKey(5) & 0xFF == 27:
      break
# print(mp_holistic.HAND_CONNECTIONS)
cv2.destroyAllWindows()
cap.release()
print(face_names)

Open
Open
Open
Open
Open
Open
Open
Open
Open
Open
Open
Open
Close
Close
Close
Close
Close
Close
Close
Close
Close
Close
Close
Close
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
Pointer
Pointer
Pointer
Pointer
Pointer
Pointer
Pointer
Pointer
Pointer
Pointer
Pointer
Pointer
Pointer
Open
Pointer
Pointer
Pointer
Open
Pointer
Pointer
Pointer
Pointer
Pointer
Pointer
Pointer
Pointer
Pointer
Pointer
Pointer
Pointer
['Rewant']
